# D4: Inference

In this notebook, we'll use statistical approaches to attempt to determine what countries the best soccer (football, for the rest of the world) players come from. Some argue that Spain produces the best footballers, while others argue it's got to be France. (And other have different opinions still.) 

We'll investigate this question using data from the video game FIFA 2020. If you're not familiar, this is a soccer video game where real pro players are encoded into the video game with stats that are supposed to accurately reflect the players in the real world.

Run the following cell. These are all you need for the assignment. Do not import additional packages.

In [ ]:
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 104)
import statsmodels
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.stats import ttest_ind
# Note: the statsmodels import may print out a 'FutureWarning'. That's fine.

## Part 1: Data & Wrangling

Fixing messy data makes up a large amount of the work of being a data scientist. 

The real world produces messy measurements and it is your job to find ways to standardize your data such that you can make useful analyses out of it. 

In this section, you will learn, and practice, how to successfully deal with unclean data.

### 1a) Load the data
Import datafile `players.csv` into a DataFrame called `df`.

Note these data come from [SOFIFA](https://sofifa.com/). Similar data have been scraped and shared on [Kaggle](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_20.csv); however the data we'll use here is distinct from that dataset.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Check out the data
df.head(5)

In the rest of Part 1, we will work on writing code, organized into functions that will allow us to transform similar respones into the same value. We will call this process: standardizing the data. 

The cell below **provides an example for the kind of code you will need to write to answer this question**. This example is separate from our actual data, and is a potential function we might use to standardize messy data - in this case, hypothetical data to the question 'What is your favourite major python version?'. 

Note some things used in this example that you need to use to standardize data:
- string methods, such as `lower` and `strip` to transform strings
- the `replace` string method, to replace a set of characters with something else
- if/else statements that check what's in our string (number, letters, etc)
- type casting, for example using `float()` to turn a variable into a float
- using `np.nan` (which stands for 'not a number') to denote missing or unknown data

The height column of the dataset is given and it is represented in either inches or centimeters. But, the data type is a string data type, the task here is to extract the number (in centimeters) and store it in the height column. The function given below `standardize_height` performs this task.

After a quick glance at the dataset, you would know that there are 4 possible ways the height column is filled as:
- It is represented in inches followed by `in` or `inches` to convey that the number is in inches.
- It is represented in centimeters followed by `cms` or `centimeters` to convey that the number is in centimeters.

The following is what the function does:
- Convert all the characters to lower case using `lower'
- Drop all white spaces using `strip`
- Check if it is inches, if so replace the keywords as empty strings and then convert the string datatype to float and the convert it to centimeters and finally return this float
- The same operation is as above is done, but the conversion from inches to centimeters isn't required here.

A thing to note here :
- The conditions for `inches` and `in` is checked simultaneously, though it could be done seperately (that is totally acceptable, this function is written to point out some common errors). Note that `replace` is used to replace `inches` first and then `in`, if it were the other way around, if the string contains `inches`, `in` would be replaced by an empty string and we would be left with `ches`, the conversion wouldn't be successful. 


In [ ]:
def standardize_height(str_in):
    
    try:
        str_in = str_in.lower()
        str_in = str_in.strip()

        if 'in' in str_in or 'inches' in str_in:
            str_in = str_in.replace('inches', '')
            str_in = str_in.replace('in', '')
            output = float(str_in)
            output = 2.54 * output
        elif 'cms' in str_in or 'centimeters' in str_in:
            str_in = str_in.replace('cms', '')
            str_in = str_in.replace('centimeters', '')
            output = float(str_in)
        else:
            output = np.nan
    except:
        output = np.nan
        
    return output

In [ ]:

assert standardize_height('180 cms') == 180
assert np.isclose(standardize_height('70 inches'), 177.8)
assert np.isnan(standardize_height('This shoud return nan :)'))


We then have to apply the transformation using the function we just defined.

In [ ]:
df['height'] = df['height'].apply(standardize_height)

In [ ]:
# use this cell to make sure that things look the way you expect

# for example
df.head(5)

### 1b) Standardize 'weight' function

Next let's check the 'weight' column. 

Check the different responses received for weight, including how many of each response we have

In [ ]:
# run this to see different weight input data
df['weight'].value_counts()

Using a similar approach to what we used for 'height', you'll write a `standardize_weight` function.

To do this you'll:
- `try` to:
    - convert all text to lowercase
    - use the string method `strip()` to remove leading and trailing characters from the gender value
    - use an `if/elif/else` to:
        - check if 'pounds' or 'lbs' exists. If so replace these words with an empty string, convert it to float, then to kilograms, and return it.
        - check if 'kgs' or 'kilograms' exists. If so replace these words with an empty string and convert it to float and return this value.
        - return `np.nan` otherwise.
- `except`: return `np.nan`
    
Note: use `value in kilograms = 0.453592 * value in pounds`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
### use this cell to test your code to make sure it does what you intend

## for example
standardize_weight('200 lbs')

In [ ]:

assert np.isclose(standardize_weight('200 Pounds'), 90.7, 0.1)
assert standardize_weight('80.0 kgs') == 80
assert np.isclose(standardize_weight('200 lbs'), 90.7, 0.1)


In [ ]:
# TESTING CELL, PLEASE DO NOT EDIT

### 1c) Transform 'weight' column

Apply the transformation, meaning, use your function and standardize weight in `df`, and assign the output back to the 'weight' column.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# use this cell to test if the resulting dataframe looks like what you want

# for example
df.head(5)

In [ ]:
# Check the results are what you expect

# for example
df['weight'].unique()

In [ ]:
# Testing cell, do not edit


## Part 2: EDA

Now that our data are ready to go, let's understand a bit more about our data...

### 2a) Nationality

We're ultimately going to be comparing players from different countries...so we should probably get a sense of **how many different nationalities are represented in our dataset**.  Use whatever method you choose to figure this out and encode the answer in a variable called `n_nats` 

In [ ]:
# determine how many different nationalities
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print(n_nats)
assert type(n_nats) == int


We're most interested in the countries that produce the most/best players, so let's generate a barplot of the nations that produce the top 10 most players and save this plot in a variable called `plot_top10`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert plot_top10

In [ ]:
# HIDDEN TEST CELL, PLEASE DO NOT EDIT

From the plot you generated, you should see that England is the most common nationality, but that France and Spain (our countries of interest) are in the top 10.

### 2b) Pairplot

In exploratory data analysis, it's often helpful to visualize relationships between multiple numerical variables to detect trends, clusters, or correlations. One powerful tool for this is `seaborn.pairplot()`.

The `pairplot` function creates a **grid** of scatter plots, where:

Each cell shows the relationship between two variables.
The diagonal typically shows histograms or self-plots of a single variable.
This plot is especially useful to identify correlations, outliers, or multivariate patterns at a glance.

Using the `pairplot`, create a pairwise plot for the following five numeric columns in our DataFrame `df`:
- `'age'`
- `'height'`
- `'weight'`
- `'potential'`
- `'overall'`

Assign the results to a variable named `fig`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert type(fig)==sns.axisgrid.PairGrid

In [ ]:
# TESTING CELL, PLEASE DO NOT EDIT

From this plot, you should get a sense of the range/possible values for each variable and see that potential and overall are certainly related to one another, as are height and weight. However, as we would expect, age is not closely related to any of the other variables. 

What do you think the variables "overall" and "potential" represent?  Do you notice a particular aspect of their relationship as well as their names that suggest to you what's going on here?

### 2c) Can we use normally distributed, equal variance statistical inference?

In the next section we are going to analyze height and weight of players from different countries.  The inference methods (such as Student's t test and ANOVA) rely on data being close to normally distributed and close to equal variance.  Let's make sure that they are!

Let's check the summary stats below. Do the following:
- subset the dataframe to only the top 4 nationalities: England, Germany, Spain, and France.
- groupby nationality
- subset the columns to only look at height and weight
- use `.agg(['mean','var','skew'])` to get relevant summary stats

*No need to assign the resulting table to a variable here.*

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

As a generic rule of thumb, we can probably get away with 
- one variance being 2x as big as another or possibly up to 4x if you like living dangerously
- skewness up to a magnitude (absolute value, we don't care about sign) of 0.5 or possibly even 1.0 if you like living dangerously


Let's ask you a quick question you can answer from the table above and the rules of thumb we gave you:

Do the values above indicate we can use tools like t-test and ANOVA that rely on normally distributed-ish, equal variance-ish groups?

Set `normal_equalvar = True` if you think yes.  And set the variable to `False` if you think no.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert normal_equalvar

NOTE: In the future if you think that you are working on a problem where the variances aren't equal, you should use Welch's method to fix the unequal variance.   And if your data is highly skewed you should either (a) use a data transformation (like log or Box-Cox) to unskew it or (b) simply use a nonparametric version of whatever statistical test you were trying to do.

If you'd like to you can add some cells here and make histograms and explore what these variance and skewness values mean in terms of your eyeball-meter.

## Part 3: Pairwise Analysis

For the first go at this we're only going to focus in on players from two countries: France and Spain and we'll answer the following questions:

1. Do the heights of players differ between Spain and France?
2. Do the weights of players differ between Spain and France?


### 3a) Subset Data

Obtain a subset of the original dataframe `df`, including only players whose `nationality` is either 'Spain' or 'France'. Store this in `df_sub`

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:

assert df_sub.shape == (2019, 15)


### 3b) Weight

Extract the weight data for all French players, storing it in `w_france`.

Extract the weight data for all Spanish players, storing it in `w_spain`.

In [ ]:
w_france = df[df['nationality'] == 'France']['weight']
w_spain = df[df['nationality'] == 'Spain']['weight']

### 3c) Difference in weight?

Carry out a t-test (using the `scipy.stats.ttest_ind()` function) to compare the two distributions. Store the test statstic in the variable `t_val` and the p-value in the variable `p_val`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print(t_val, p_val)


I recommend that you report p-values directly and avoid making statements about whether something is "statistically significant" at a given threshold. This is usually the better approach to analysis when you haven't worked extensively in a field. 

But some kinds of science/engineering practice do have established stadards of evidence that are expected when you work in those fields. Whether thats $\alpha=0.05$ (a very lax standard) or something more stringent like we are doing below.  

Recall that $\alpha=0.05$ means that there's a 1/20 chance for this data to be observed if we assume the null hypothesis is true.  As we said in lecture p-values are $P(D|H_0)$ and not the probability of a false positive result which would be $P(H_0 | D)$.  Without more information, we can't know exactly what the false positive rate will be for a given p-value.  But research suggests that in practice a $0.05$ threshold might actually be approaching 50% false positive rate.

So in practice many branches of science or engineering may have rules of thumb for p-values that are much more stringent than 0.05... like we use below

In [ ]:
# Check if statistical test passes significance
# using an alpha value of 0.001. This code provided.
if p_val < 0.001:
    print('There is a significant difference!')
else:
    print('There is NOT a significant difference!')

### 3d) Height

Lets do the same thing again, but this time t-test if the French and Spanish players differ in height.

Store height values in `h_france` and `h_spain`.  Then use those variables to construct a `ttest_ind()` and store the outcome of the test in variables `t_val_h` and `p_val_h`. 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print(t_val_h, p_val_h)
assert type(h_france) == pd.Series
assert type(h_spain) == pd.Series

In [ ]:
# Check if statistical test passes significance
# using an alpha value of 0.001. This code provided.
if p_val_h < 0.001:
    print('There is a significant difference!')
else:
    print('There is NOT a significant difference!')

### 3e) Who's taller and does it matter?

In 3d, you should have determined that there is a height difference between French and Spanish soccer players...but who's taller? Determine the difference in means here, storing the mean height for each country in `mean_france` and `mean_spain`, respectively.  Also store the standard deviation of each country in `std_france` and `std_spain`

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
print(f'French player  mean {mean_france:.2f}cm\nSpanish player mean {mean_spain:.2f}cm')
print(f'Difference of means {mean_france - mean_spain:.2f}cm')
print(f'French player  std {std_france:.2f}cm\nSpanish player std {std_spain:.2f}cm')

def cohen_d_independent(x, y):
    n1, n2 = len(x), len(y)
    mean1, mean2 = np.mean(x), np.mean(y)
    var1, var2 = np.var(x, ddof=1), np.var(y, ddof=1) # ddof=1 for sample variance

    pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    cohens_d = (mean1 - mean2) / pooled_std
    return cohens_d, pooled_std
    
cod, pooled_std = cohen_d_independent( h_france, h_spain) 
print(f'Pooled std of france and spain {pooled_std:.2f}')
print(f'Cohen\'s d measure of effect size (difference of means / pooled std) {cod:.2f}')

# note there are some hidden tests here to make sure you calculated things correctly

Now that you've seen 
- the p value of mean height differences
- the actual mean height differences in cm
- and the Cohen's d effect size of those heights differences

And I'd like to add these following rules of thumb
 - p values can be thought of as no evidence ($p \ge 0.05$), weak evidence ($0.05 > p \ge 0.001$) and strong evidence ($p < 0.001$)
 - Cohen's d can be though of as minimal effect ($ d \le 0.2$), weak effect ($ 0.2 < d \le 0.5$), medium effect  ($ 0.5 < d \le 0.8$) and strong effect ($d > 0.8$)
 - note that these are some BS rules of thumb, don't take them too seriously
    
Using this infor lets ask the following questions:


Is there no/weak/strong evidence that there is a true height difference between these nationalities?  Is there a minimal/weak/medium/strong effect of nationality on player height?  

To give us your answer, define a variable `evidence` and assign it a string of either 'no', 'weak', or 'strong', and then define a variable called `effect` and assign it a string of 'minimal', 'weak', 'medium', or 'strong'

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:

assert evidence
assert evidence.lower() in ['no','weak','strong']
assert effect
assert effect.lower() in ['minimal','weak','medium','strong']


In [ ]:
# TESTING CELL, PLEASE DO NOT EDIT

### 3f) What about other nationalities?

Now that we investigated the heights of players in France and Spain, let's investigate the heights of players from other countries. 

Using this list of nationalities: ```nats = ['Japan','Spain','England','France','Germany','Netherlands']```, create a dataframe that filters the nationalities to be only those in `nats`. Keep the dataframe in a variable called `df_nats`.

In [ ]:
nats = ['Japan','Spain','England','France','Germany','Netherlands']

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Testing cell, please do not edit


Now that we filtered the data, plot a box plot to visualize the distribution of heights of players across the selected nationalities in `df_nats`. Color each box plot by `nationality`.

Now at this point you will almost certainly encounter a situation where the automatic legend is obscuring important parts of the plot.  This is a common problem and there is no magic automatic solution to put text on a plot that always works. 

Take a look at the docs for legend placement in matplotlib: https://matplotlib.org/stable/users/explain/axes/legend_guide.html#legend-location and particularly the part about `bbox_to_anchor` argument which allows you to force the legend to be outside the plotting area.

Exactly how you choose to use it is a matter of your own personal aesthetics... but I found that I liked `ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1))` for a horizontal boxplot.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert (ax.get_xlabel().lower() == 'height') or (ax.get_ylabel().lower() == 'height')
legend = ax.get_legend()
assert legend



In our previous t-tests, we compared players from two different nationalities: France and Spain. Student's t-test is made to compare two groups.

But here we have 6 different nationalities. How many different t-tests can be run with these set of nationalities to examine differences in height between pairs of countries?  Note that a France-England t-test is the same as an England-France t-test and you should only count this combination only once.

Save your answer as a variable named `combination_t_test` 

A) 5

B) 6

C) 15 

D) 25

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert combination_t_test.upper() in ['A', 'B', 'C', 'D']
assert type(combination_t_test) == str


When we have multiple t-tests looking at the same question there is a higher risk of getting a false positive result than when we only run one t-test.  When we have multiple stastical tests we often think about the _family-wise_ error rate.  That is, think about the p value as representing the probability that AT LEAST ONE of the $n$ tests cannot reject the null hypothesis, implying that the observed difference between AT LEAST ONE pair of groups is actually due to chance alone.  

A family of $n$ t-tests yields a family wise error rate of $p= 1 - \prod_{i=1}^n (1 - p_i)$... which gets bad quickly as the number of tests grows. If we have 10 tests all at $p=0.05$ that means the family-wise p value would be $p=0.40$

There are methods to correct for _multiple comparisons_ (that's the jargon term for doing a bunch of pairwise statistical tests), such as Bonferroni or Benjamini-Hochberg (feel free to look those up).  Using one of these methods lets you calculate p-values that represent the family wise error rate.

But even better may be to simply use a test that is directly meant to compare in a family-wise fashion.  Instead of asking "for all pairs of countries, is there any difference between each pair?" (multiple t-tests) we can ask "is there any difference between countries?".  Now a family wise test like this will NOT tell us which countries are different, but it will tell us if there's any differences among the countries.

The family-wise test that matches our situation is the one-way Analysis of Variance (ANOVA). A "regular" one-way ANOVA has a similar set of assumptions about the data as we see for Student's t, but it can be used for any number of groups, not just 2.

For every group in the data, ANOVAs compare the ratio of between-group variance to the within-group variance. When most the variance is due to the grouping, then the groups matter and explain the values we observe.  The test uses the F-statistic, which is converted into a p-value in the same way that the t-statistic if converted into a p-value when we use Student's t.  

Let's use a library called statsmodels to do our ANOVA.  We will use for a few other things next week too.  Here's a demo of what that looks like:





In [ ]:
import statsmodels.api as sm

anova_result = sm.stats.anova_oneway(data=df_nats['height'],groups=df_nats['nationality'])

print(anova_result)

From this run we can see some intersting results.  You can see what all the group means and variances are.  you can see that the ANOVA function has decided that the variances among groups are different enough to try to correct for that using the Welch method.  You can see that the p value indicates that we are very certain there is a difference somewhere among all the countries.  

And that p value result should be very unsurprising... after all we already know that the France and Spain have different heights according to a t-test.

Now assuming that we had never run that t-test beforehand we would be in an uncomfortable spot.  We know somewhere in the countries there are height differences, but we do not know which countries are different from which others.  

After a family wise test like an ANOVA you can undertake a _post hoc_ (Latin: after the fact) test.  Once we know there's a difference, we can use a bunch of pairwise tests to decide which countries are different from which others.  These methods correct for the multiple comparisons involved, so the p-values for each comparison should be understood as accurately representing the probability of the result under a null hypothesis but accounting for the increased error rate of doing this kind of thing repeatedly.

Most commonly after an ANOVA we use Tukey's Honestly Significant Differences (HSD) method. You can find the documentation for statsmodels HSD implementation here https://www.statsmodels.org/dev/generated/statsmodels.stats.multicomp.pairwise_tukeyhsd.html#statsmodels.stats.multicomp.pairwise_tukeyhsd

We can see that the groupings ran individual ANOVA tests in Tukey HSD. With each row representing a combination of nationalities to investigate, we can determine which differences ("meandiff") are statistically significant using the "p-adj" column which contains the multiple comparison adjusted p values. The "lower" and "upper" bounds are the 95% confidence interval on the true difference between group means.  The "reject" column is True if $p<\alpha$, and it when it is false note that both $p>=\alpha$ and also the confidence interval will include $0.0$ inside its range


In [ ]:
tukey_result = statsmodels.stats.multicomp.pairwise_tukeyhsd( df_nats['height'], 
                                                             groups = df_nats['nationality'], 
                                                             alpha = 0.05)
print(tukey_result)

The Tukey HSD produced all versions of comparing two groups out of the 6 nationalities. Take a note to see which comparison had statistically significant differences in their heights. 

In the table, `group1` is one group, `group2` is the other group for comparison, `meandiff` is the mean difference of height between the two groups, `p-adj` is the p-value from post hoc comparison through ANOVA. `lower` and `upper` is the upper and lower bound of the confidence interval while `reject` is if the null hypothesis would be rejected.

Let's take a look at the findings that the table shows. Fill in the blanks for these sentences to understand the results from each test. 

**Question 1**
In the unobserved population, the difference between the mean heights of English and German players in the dataset is __ cm.  Now according to the 95% confidence interval the true populations' mean height difference may actually be lower.  The lowest magnitude it could be is __ cm.

NB: give this answer as a tuple or list called `answer_1`, with the first element containing the numeric mean difference value, and the second element containing the numeric lower CI bound.

**Question 2**
Looking at the ANOVA test for Netherlands and Germany, the p_value of the test was _____, which means that p > 0.01. Therefore we can _____ (fail to reject / reject) the null hypothesis.

NB: give this answer as a tuple or list called `answer_2`, with the first element containing a numeric p value, and the second element containing either the string 'fail to reject' or the string 'reject'

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert type(answer_1) in (tuple, list)
assert len(answer_1)  == 2
assert type(answer_1[0]) == float
assert type(answer_1[1]) == float


assert type(answer_2) in (tuple, list)
assert len(answer_2)  == 2
assert type(answer_2[0]) == float
assert type(answer_2[1]) == str


## End of Discussion Lab Exercise #3

When you think you're done, verify that everything is working correctly by doing the following:
1. Save this notebook
2. Choose the pull down menu item Kernel -> Restart and Run All Cells
3. Check that you see the print statement in the cell below. If you see that happy sunglasses face emoji, it means that you have passed all the publicly visible tests. Yay!! 

In [ ]:
print("\N{SMILING FACE WITH SUNGLASSES}")